In [1]:
print("hello")

hello


## project started 
Demo time

In [ ]:
import os
import requests
from dotenv import load_dotenv

from IPython.display import Markdown, display
from openai import OpenAI

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
model="gpt-4o-mini"
openai = OpenAI()

In [5]:
system_prompt = """
You are an expert disaster risk advisor specializing in cloudburst and flood events.
You will analyze weather and terrain conditions (rainfall, humidity, pressure, soil moisture, slope, river proximity, etc.)
and assess the likelihood of floods.

Your response must always include:
1. Flood Risk Level → Low / Moderate / High / Severe
2. Cloudburst Probability → Yes / No with reasoning
3. Key Factors → List the main conditions driving the assessment
4. Recommendations → Clear and actionable steps for residents and authorities

Keep the language simple and practical so that even non-technical people can understand.
"""


In [6]:
user_prompt = """
Here are the observed conditions:

- Rainfall last 24h: 120 mm
- Forecast next 6h: 80 mm
- Soil moisture: 85%
- Slope: Steep
- River proximity: 200 m
- Flow rate: High
- Pressure: 1005 hPa (falling)
- Cloud cover: 95%

Question: Is there a chance of flood or cloudburst, and what should residents and authorities do?
"""
    

In [7]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [8]:
def call_gpt(messages):
    response = openai.chat.completions.create(model=model, messages=messages)
    print(response.choices[0].message.content)

In [9]:
#  trial
call_gpt(messages)

1. **Flood Risk Level**: **Severe**

2. **Cloudburst Probability**: **Yes**. The heavy rainfall of 120 mm already observed, combined with an additional 80 mm forecasted, indicates a high likelihood of rapid water accumulation, especially on steep slopes. The saturated soil and high humidity further increase the risk of a sudden downpour, characteristic of cloudbursts.

3. **Key Factors**:
   - **Recent Rainfall**: 120 mm in the last 24 hours – significant and likely overwhelming drainage systems.
   - **Forecasted Rainfall**: 80 mm expected in the next 6 hours – this would exacerbate the conditions.
   - **Soil Moisture**: At 85%, the soil is highly saturated and unable to absorb any more water.
   - **Slope**: Steep terrain increases runoff and the speed at which water can flow, raising the flood risk.
   - **River Proximity**: 200 m from a water body, which is likely to swell and potentially overflow.
   - **Flow Rate**: High flow rates indicate already elevated water levels in nearb

In [ ]:
def setup_vectorstore(db_name="vector_db"):
    """Setup connection to existing vector database"""
    try:
        embeddings = OpenAIEmbeddings()
        vectorstore = Chroma(
            persist_directory=db_name, 
            embedding_function=embeddings
        )
        print(f"Connected to vector database with {vectorstore._collection.count()} documents")
        return vectorstore
    except Exception as e:
        print(f"Error connecting to vector database: {e}")
        return None


In [ ]:

def get_relevant_context(vectorstore, query, k=3):
    """Get relevant documents from vector database"""
    if vectorstore is None:
        return ""
    
    try:
        docs = vectorstore.similarity_search(query, k=k)
        
        context = "\n\nRELEVANT CONTEXT FROM KNOWLEDGE BASE:\n"
        for i, doc in enumerate(docs, 1):
            doc_type = doc.metadata.get('doc_type', 'Unknown')
            context += f"\n{i}. From {doc_type}:\n{doc.page_content[:400]}...\n"
        
        return context
    except Exception as e:
        print(f"Error retrieving context: {e}")
        return ""

In [ ]:
def create_enhanced_system_prompt():
    return """
    You are an expert disaster risk advisor specializing in cloudburst and flood events in Uttarakhand, India.
    You will analyze weather and terrain conditions along with relevant historical and geographical context.
    
    Use the provided context from the knowledge base to inform your risk assessment.
    Reference specific historical events, geographical characteristics, and past patterns when relevant.
    
    Your response must always include:
    1. Flood Risk Level → Low / Moderate / High / Severe
    2. Cloudburst Probability → Yes / No with reasoning
    3. Key Factors → List the main conditions driving the assessment
    4. Historical/Geographical Context → Reference relevant information from the knowledge base
    5. Recommendations → Clear and actionable steps for residents and authorities
    
    Keep the language simple and practical so that even non-technical people can understand.
    """

In [ ]:
def call_gpt_with_context(messages, vectorstore=None, location=None):
    """Call GPT with contextual information from vector database"""
    
    # Extract user prompt for context search
    user_content = None
    for msg in messages:
        if msg["role"] == "user":
            user_content = msg["content"]
            break
    
    # Get relevant context
    if vectorstore and user_content:
        search_query = user_content
        if location:
            search_query = f"{location} {search_query}"
        
        context = get_relevant_context(vectorstore, search_query)
        
        # Add context to user message
        for msg in messages:
            if msg["role"] == "user":
                msg["content"] = msg["content"] + context
                break
    
    # Call GPT
    response = openai.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content

In [ ]:









# Initialize vector database connection
vectorstore = setup_vectorstore("vector_db")

# Update your system prompt
enhanced_system_prompt = create_enhanced_system_prompt()

# Your existing user prompt with location information
user_prompt_with_location = """
Location: Uttarkashi District

Here are the observed conditions:
- Rainfall last 24h: 120 mm
- Forecast next 6h: 80 mm
- Soil moisture: 85%
- Slope: Steep
- River proximity: 200 m
- Flow rate: High
- Pressure: 1005 hPa (falling)
- Cloud cover: 95%

Question: Is there a chance of flood or cloudburst, and what should residents and authorities do?
"""

# Create messages with enhanced system prompt
enhanced_messages = [
    {"role": "system", "content": enhanced_system_prompt},
    {"role": "user", "content": user_prompt_with_location}
]

# Call with context
print("=== ENHANCED RISK ANALYSIS WITH CONTEXT ===")
enhanced_result = call_gpt_with_context(enhanced_messages, vectorstore, "Uttarkashi")
print(enhanced_result)

# Cell 7: Function for quick district queries
def query_district_info(district_name, vectorstore):
    """Quick function to get district-specific information"""
    if not vectorstore:
        return "Vector database not available"
    
    query = f"{district_name} district characteristics risk factors historical events"
    docs = vectorstore.similarity_search(query, k=5)
    
    if not docs:
        return f"No information found for {district_name}"
    
    result = f"=== {district_name.upper()} DISTRICT INFORMATION ===\n\n"
    for i, doc in enumerate(docs, 1):
        doc_type = doc.metadata.get('doc_type', 'Unknown')
        result += f"{i}. {doc_type}:\n{doc.page_content[:300]}...\n\n"
    
    return result

# Cell 8: Example district query
print("=== DISTRICT INFORMATION ===")
district_info = query_district_info("Rudraprayag", vectorstore)
print(district_info)

Connected to vector database with 135 documents
=== ENHANCED RISK ANALYSIS WITH CONTEXT ===
1. **Flood Risk Level**: **Severe**  
   The observed and forecasted rainfall levels, high soil moisture, and steep slopes create conditions conducive to severe flooding.

2. **Cloudburst Probability**: **Yes**  
   The combination of high rainfall (120 mm in the last 24h, with an additional 80 mm forecast), steep slopes, and moist soil conditions significantly increases the likelihood of a cloudburst event. The high cloud cover (95%) further supports this risk.

3. **Key Factors**:  
   - Recent rainfall accumulation (120 mm in 24h).
   - High forecast rainfall (80 mm in the next 6h).
   - High soil moisture (85%).
   - Steep terrain, increasing runoff.
   - Proximity to river, only 200 m away, with high flow rates.

4. **Historical/Geographical Context**:  
   Uttarkashi district is recognized as a high-risk area for cloudbursts due to its complex topography and significant variations in rainf